# Variables Demograficas

- [ ] El tener multiples caracterisiticas demograficas permite hacer cortes y el sesgo...

In [2]:
import os
import re
import ast
import pandas as pd

import helpers.preprocess_text as pre_text

import helpers.preprocess_location as pre_loc

## Geolocalización


### Objeto geo del tweet

In [3]:
# Cargar tweets del dataset ejemplo
df_tweets = pd.read_json("./files/1_dataset.jsonl", lines=True)
df_tweets = df_tweets[['id', 'author_id', 'geo']]

# Cargar los objetos anidados geo
df_geo = pd.DataFrame(list(df_tweets['geo']))
df_geo = df_geo.drop(columns=['id'])
# df_geo.columns = list(map(lambda x: f"geo_{x}", df_geo.columns))

df_tweets = df_tweets.drop(columns=['geo'])
df_tweets = df_tweets.join(df_geo)

df_tweets

,id,author_id,place_id,country_code,name,country,place_type,geo,full_name,coordinates
0,1568384770672934912,2671423238,00a8d523465806f4,AR,Colón,Argentina,city,"{'type': 'Feature', 'bbox': [-64.122293, -31.0...","Colón, Argentina",NaN
1,1568384665731440640,1361302057945333760,666f0989b9fd5616,CO,Antioquia,Colombia,admin,"{'type': 'Feature', 'bbox': [-77.1214380001307...","Antioquia, Colombia",NaN
2,1568357940461191168,1391458758459600896,de24fd487f41bad3,MX,Iztacalco,México,city,"{'type': 'Feature', 'bbox': [-99.137171, 19.37...","Iztacalco, Distrito Federal",NaN
3,1568337671931691008,1149464602452877312,018f1cde6bad9747,AR,Ciudad Autónoma de Buenos Aires,Argentina,city,"{'type': 'Feature', 'bbox': [-58.5317922, -34....","Ciudad Autónoma de Buenos Aires, Argentina",NaN
4,1568333644934135808,119160105,00f84d414936f28e,AR,Córdoba,Argentina,city,"{'type': 'Feature', 'bbox': [-64.3195, -31.523...","Córdoba, Argentina","{'type': 'Point', 'coordinates': [-64.1847387,..."
...,...,...,...,...,...,...,...,...,...,...
8764,1477088691877261312,397706715,0105a80417c159b4,CO,Barranquilla,Colombia,city,"{'type': 'Feature', 'bbox': [-74.9257650027657...","Barranquilla, Colombia",NaN
8765,1477088689901682688,397706715,0105a80417c159b4,CO,Barranquilla,Colombia,city,"{'type': 'Feature', 'bbox': [-74.9257650027657...","Barranquilla, Colombia",NaN
8766,1477086472469684224,1176580744036196352,7262f97768b53492,MX,Huixtla,México,city,"{'type': 'Feature', 'bbox': [-92.7213064, 14.9...","Huixtla, Chiapas",NaN
8767,1477067319830478848,579360940,018f1cde6bad9747,AR,Ciudad Autónoma de Buenos Aires,Argentina,city,"{'type': 'Feature', 'bbox': [-58.5317922, -34....","Ciudad Autónoma de Buenos Aires, Argentina",NaN


- [ ] Describir la funcion

In [4]:
def normalize_twitter_geolocation(df_data):

    # TODO: Make this a wrapper(?)
    df_countries = pd.read_csv(f"./resources/countries_data.csv",
                               usecols=['ISO_2', 'ISO_3', 'name_short_es'],
                               sep='\t')

    ISO_codes = dict(df_countries.iloc[:, :2].values)
    countries_names = dict(df_countries.iloc[:, [0, 2]].values)

    # Normalize author country data
    srs_country_codes = df_data['country_code']
    df_data['country_code'] = srs_country_codes.map(ISO_codes)
    df_data['country_name'] = srs_country_codes.map(
        countries_names)

    # Normalize federal divisions
    admin_index = (df_data['place_type'] == 'admin')
    df_data['federal_division'] = df_data[admin_index]['name']

    # Normalize cities
    city_index = (df_data['place_type'] == 'city')
    df_data['city'] = df_data[city_index]['name']
    
    # Get bbox centroid
    df_data['geo_type'] = df_data['geo'].apply(lambda x: x['type'])
    df_data['geo_bbox'] = df_data['geo'].apply(lambda x: x['bbox'])
    df_data['latitude'] = df_data['geo_bbox'].apply(lambda x: (x[1]+x[3])/2)
    df_data['longitude'] = df_data['geo_bbox'].apply(lambda x: (x[0]+x[2])/2)

    return df_data

In [5]:
df_ids = df_tweets[['id', 'place_id']]
df_ids

,id,place_id
0,1568384770672934912,00a8d523465806f4
1,1568384665731440640,666f0989b9fd5616
2,1568357940461191168,de24fd487f41bad3
3,1568337671931691008,018f1cde6bad9747
4,1568333644934135808,00f84d414936f28e
...,...,...
8764,1477088691877261312,0105a80417c159b4
8765,1477088689901682688,0105a80417c159b4
8766,1477086472469684224,7262f97768b53492
8767,1477067319830478848,018f1cde6bad9747


In [6]:
df_geolocs = df_tweets.iloc[:, 1:].dropna(subset=['place_id']) \
                      .drop_duplicates(subset=['place_id'])
df_geolocs = normalize_twitter_geolocation(df_geolocs)
df_geolocs = df_geolocs[['place_id', 'country_code', 
                         'country_name', 'federal_division', 'city', 
                         'latitude', 'longitude']]
df_geolocs

,place_id,country_code,country_name,federal_division,city,latitude,longitude
0,00a8d523465806f4,ARG,Argentina,NaN,Colón,-31.000023,-64.086213
1,666f0989b9fd5616,COL,Colombia,Antioquia,NaN,7.139205,-75.508819
2,de24fd487f41bad3,MEX,México,NaN,Iztacalco,19.399101,-99.096496
3,018f1cde6bad9747,ARG,Argentina,NaN,Ciudad Autónoma de Buenos Aires,-34.604315,-58.442643
4,00f84d414936f28e,ARG,Argentina,NaN,Córdoba,-31.415457,-64.198026
...,...,...,...,...,...,...,...
8742,01e2d9bb7bfd4799,COL,Colombia,NaN,Belalcázar,4.977568,-75.863214
8747,00b62c7d4e24b866,ARG,Argentina,NaN,San Lorenzo,-32.749032,-60.746994
8753,7fb5c5b050f42ca5,COL,Colombia,Cundinamarca,NaN,4.766092,-73.995244
8756,25904a4dfba77093,MEX,México,NaN,Ezequiel Montes,20.653227,-99.860001


In [8]:
df_geo = df_ids.merge(df_geolocs, on='place_id', how='left')
df_geo = df_geo[~df_geo['country_code'].isna()]
df_geo

,id,place_id,country_code,country_name,federal_division,city,latitude,longitude
0,1568384770672934912,00a8d523465806f4,ARG,Argentina,NaN,Colón,-31.000023,-64.086213
1,1568384665731440640,666f0989b9fd5616,COL,Colombia,Antioquia,NaN,7.139205,-75.508819
2,1568357940461191168,de24fd487f41bad3,MEX,México,NaN,Iztacalco,19.399101,-99.096496
3,1568337671931691008,018f1cde6bad9747,ARG,Argentina,NaN,Ciudad Autónoma de Buenos Aires,-34.604315,-58.442643
4,1568333644934135808,00f84d414936f28e,ARG,Argentina,NaN,Córdoba,-31.415457,-64.198026
...,...,...,...,...,...,...,...,...
8764,1477088691877261312,0105a80417c159b4,COL,Colombia,NaN,Barranquilla,10.958073,-74.849933
8765,1477088689901682688,0105a80417c159b4,COL,Colombia,NaN,Barranquilla,10.958073,-74.849933
8766,1477086472469684224,7262f97768b53492,MEX,México,NaN,Huixtla,15.125592,-92.563117
8767,1477067319830478848,018f1cde6bad9747,ARG,Argentina,NaN,Ciudad Autónoma de Buenos Aires,-34.604315,-58.442643


### Atributo *location* del usuario

## Asignar género

El poder asignar un género a los tweets nos permite hacer un estudio de como los diferentes géneros abordan el tema de estudio, en este caso el de Migración. Twitter no recolecta esa información al hacer la cuenta, por lo que no podemos encontrar un atributo dado por la API, pero al igual que la geolocalización, podemos inferirla a partir de atributos como el nombre, la descripción o incluso de la foto de perfil.

En esta notebook se van a revisar algunas opciones para asignar género al autor:
- Buscar nombres típicamente masculinos o femeninos en los campos de username, descripción
- Buscar pronombres (She/Her o He/His) en la descripción del usuario
- Buscar la nacionalidad (e.g.: Mexicano / Mexicana) en los campos descripción

También se podría trabajar con adjetivos, profesiones, oficios o títulos.

Nota: Dado que estos campos son abiertos y el nivel de veracidad recae completamente en cada usuario, se puede esperar que estos métodos tengan falsos positivos o negativos. Igualmente, tampoco se puede esperar que se termine asignando un género a todos los autores.

## Cargar nombres, pronombres, nacionalidades y géneros

En el directorio llamado ./resources hay un archivo llamado names_genders.csv que contiene una columna con nombres, una con el género que típicamente tienen las personas llamadas así y una última columna de si es un nombre compuesto como; Maria Jose o Jose Guadalupe.

In [ ]:
df_names = pd.read_csv('./resources/names_genders.csv')

pre_text.remove_duplicated_chars(df_names, 'name')

# Names composed
df_names_double = df_names[df_names['is_double']]
regex_names_double = f"\\b((?:{'|'.join(df_names_double['name'])}))\\b"
regex_names_double = re.compile(regex_names_double)

# Single names
df_names_single = df_names[~df_names['is_double']]
regex_names_single = f"\\b((?:{'|'.join(df_names_single['name'])}))\\b"
regex_names_single = re.compile(regex_names_single)

# Names to gender
dict_names_gender = df_names.set_index('name')['gender'].to_dict()
df_names

Despues hay un diccionario con los posibles strings conteniendo pronombres y el genero que identifican. 

In [ ]:
dict_pronouns_gender = {
    "she her": 'F',
    "her she": 'F',
    "he him": 'M',
    "him he": 'M'
}

regex_pronous = f"\\b((?:{'|'.join(dict_pronouns_gender.keys())}))\\b"
regex_pronous = re.compile(regex_pronous)
regex_pronous

Un diccionario con las posibles nacionalidades en las diferentes variaciones segun el genero

In [ ]:
dict_nationalities_gender = {
    'mexicano': 'M',
    'mexicana': 'F',
    'colombiano': 'M',
    'colombiana': 'F',
    'argentino': 'M',
    'argentina': 'F'
}

regex_nationalities = f"\\b((?:{'|'.join(dict_nationalities_gender.keys())}))\\b"
regex_nationalities = re.compile(regex_nationalities)
regex_nationalities

## Cargar tweets

Primero hay que cargar los tweets, específicamente el ID del tweet, el ID del usuario, el nombre del autor, su descripción. Los últimos tres campos son abiertos, es decir, que el usuario puede ingresar lo que desee a excepción de algunos caracteres especiales. Esto implica que antes de todo hay que pre-procesarlos para así reducir el número de variantes para una misma nacionalidad y/o nombre.

Nota: Usaremos de ejemplo el primer dataset que se obtuvo en la notebook "2_Definiendo_Queries.ipynb".

In [ ]:
df_tweets = pd.read_json("./archivos_queries/1_dataset.jsonl", lines=True)
df_tweets = df_tweets[['id', 'author_id', 'author']]

df_tweets['author_name'] = df_tweets['author'].apply(lambda x: x['name'])
df_tweets['author_description'] = df_tweets['author'].apply(lambda x: x['description'])

df_tweets = df_tweets.drop(columns=['author'])

df_tweets

Para evitar trabajar con autores duplicados, trabajaremos solo con una copia única de los autores, asignaremos géneros y luego haremos JOIN a los tweets usando como llave la columna `author_id`.

In [ ]:
df_users = df_tweets.drop_duplicates(subset=['author_id'], 
                                  keep='last').reset_index(drop=True)
df_users = df_users.iloc[:, 1:]
df_users

## Preprocesamiento

La siguiente función se encarga de pasar todo a minúsculas, eliminar acentos, URLs, emails, números, caracteres que no sean letras y espacios en blancos duplicados. Esto nos ayudará a que al buscar los nombres o nacionalidades en los campos, se tenga más posibilidades de encontrarlos, dado que no habrá variaciones del tipo: "NOE", "NoE", "Noe", "Noé". Permitiendo que todas esas variaciones sean validas y encontradas con la cadena de texto "noe".

In [ ]:
def preprocess_for_names(df_data, new_col, flg_remove_two_chars=True):

    at_least_3_letter_reg = re.compile(r'\b\w{1,2}\b')
    at_least_3_letter_reg_capture = re.compile(r'\b(\w{2})\b')

    pre_text.initial_preprocessing(df_data, new_col,
                                   flg_remove_emojis=True,
                                   flg_lower=True)

    pre_text.remove_urls(df_data, new_col)
    pre_text.remove_emails(df_data, new_col)
    pre_text.remove_urls(df_data, new_col)
    pre_text.remove_numbers(df_data, new_col, 
                            replace_char=' ')

    pre_text.remove_duplicated_chars(df_data, new_col)
    pre_text.remove_non_alphanumeric(df_data, new_col)

    pre_text.remove_single_letters(df_data, new_col)
    
    if(flg_remove_two_chars):
        df_data[new_col] = df_data[new_col].str.replace(at_least_3_letter_reg, 
                                                        ' ', regex=True)

    pre_text.remove_multiple_blank_spaces(df_data, new_col)
    df_data[new_col] = df_data[new_col].str.strip()

    valid_texts = ((~df_data[new_col].isna()) *
                   (df_data[new_col] != '') *
                   (~df_data[new_col].str.contains(r'^\w$', regex=True)))
    df_data = df_data[valid_texts]

    return df_data

Se le aplica la función de limpieza a ambas columnas

In [ ]:
df_users = preprocess_for_names(df_users.copy(), 'author_name')
df_users = preprocess_for_names(df_users.copy(), 'author_description',
                                flg_remove_two_chars=False)
df_users

## Buscar los nombres en la columna del nombre

In [ ]:
# Copia de la tabla de usuarios
df_by_names = preprocess_for_names(df_users.copy(), 'author_name')

# Buscar nombres
df_tmp2 = df_by_names['author_name'].str.extract(regex_names_double).dropna()

df_tmp1 = df_by_names[~df_by_names.index.isin(df_tmp2.index)]['author_name']
df_tmp1 = df_tmp1.str.extract(regex_names_single).dropna()

# Asignar género en base al nombre encontrado
df_by_names['extracted_name'] = pd.concat([df_tmp1, df_tmp2])[0]
df_by_names['gender'] = df_by_names['extracted_name'].map(dict_names_gender)

df_by_names = df_by_names.dropna()
df_by_names[['author_name', 'extracted_name', 'gender']]

## Buscar los probombres en la descripción

In [ ]:
# Descartar usuarios a los que ya se les asigno un género
df_pronouns = df_users[~df_users['author_id'].isin(df_by_names['author_id'])].copy()

# Buscar los probombres en la descripción
df_pronouns['extracted_pronoun'] = df_pronouns['author_description'].str.extract(regex_pronous).dropna()

# Asignar un género en base a los pronombres encontrados
df_pronouns['gender'] = df_pronouns.dropna()['extracted_pronoun'].map(dict_pronouns_gender)

df_pronouns = df_pronouns.dropna()
df_pronouns[['author_name', 'extracted_name', 'gender']]

### Nota
Los siguientes métodos necesitan un mayor preprocesamiento del realizado en esta notebook, debido a que puede haber nombres de países que se confundan con la nacionalidad (e.g. Argentina) o los usuarios sean de una organización y/o negocio, pero mencionen a alguna nacionalidad. Se debería considerar añadir un paso de preprocesamiento donde se intente identificar que cuentas no son controladas por un solo individuo y representan a un conjunto.

## Buscar nacionalidades en la descripción

In [ ]:
# Descartar usuarios a los que ya se les asigno un género
df_nationalities = df_users.loc[~(df_users['author_id'].isin(df_by_names['author_id']) + 
                                  df_users['author_id'].isin(df_pronouns['author_id']))].copy()

# Buscar los probombres en la descripción
df_nationalities['extracted_nationality'] = df_nationalities['author_description'].str.extract(regex_nationalities).dropna()

# # Asignar un género en base a los pronombres encontrados
df_nationalities['gender'] = df_nationalities.dropna()['extracted_nationality'].map(dict_nationalities_gender)

df_nationalities = df_nationalities.dropna()
df_nationalities

## Buscar nombres en la descripción del usuario

In [ ]:
# Descartar usuarios a los que ya se les asigno un género
df_description = df_users.loc[~(df_users['author_id'].isin(df_by_names['author_id']) + 
                                df_users['author_id'].isin(df_pronouns['author_id']) + 
                                df_users['author_id'].isin(df_nationalities['author_id']))].copy()

# Extraer nombres de la descripción
df_tmp2 = df_description['author_description'].str.extract(regex_names_double).dropna()

df_tmp1 = df_description[~df_description.index.isin(df_tmp2.index)]['author_description']
df_tmp1 = df_tmp1.str.extract(regex_names_single).dropna()

# Asignar genero a los nombres
df_description['extracted_name'] = pd.concat([df_tmp1, df_tmp2])[0]
df_description['gender'] = df_description['extracted_name'].map(dict_names_gender)

df_description = df_description.dropna()
df_description

## Más ideas
- Buscar profesiones y sus abstracciones en el nombre del la cuenta. E.g.: Doctor/Dr. y Doctora/Dra.
- Buscar titulos como "Papá", "Mamá", "Coordinador", "Coordinadora"
- Buscar adjetivos calificativos en la descripción
- Identificar cuentas que representen a Secretarias Publicas, Oraganización, Clubs, Empresas, etc.
- Aplicar un NER para hacer las busquedas de manera más inteligentes y focalizada

## Añadir la columna gender a cada tweet

Primero se juntan las tablas de los usuarios con género obtenidos con los diferentes métodos

In [ ]:
df_genders = pd.concat([df_by_names, df_pronouns, df_nationalities, df_description])
df_genders = df_genders.drop(columns=['extracted_name', 'extracted_pronoun', ])

df_genders = df_genders[['author_id', 'gender']]
df_genders.columns = ['author_id', 'author_gender']
df_genders

Se hace el merge de las tablas, el equivalente a un JOIN de SQL

In [ ]:
df_tweets = df_tweets.merge(df_genders, on='author_id', how='left')
df_tweets

TODOs
- [ ] Añadir una grafica donde compare cuantos M y F publicaron para cada pais
- [ ] Otra grafica donde solo sea un pie o barras que compare M, F y apagado aquellos con NaN